In [13]:
import wikipediaapi

In [17]:
wiki_wiki = wikipediaapi.Wikipedia('en',
        extract_format=wikipediaapi.ExtractFormat.HTML)
page_py = wiki_wiki.page('Python_(programming_language)')

In [15]:
page_py

Python_(programming_language) (id: ??, ns: 0)

In [18]:
print(page_py.text[0:5000])

<p class="mw-empty-elt">
</p>
<p><b>Python</b> is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.</p><p>Python is dynamically-typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.</p><p>Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000 and introduced new features such as list comprehensions, cycle-detecting garbage collection, reference counting, and Unicode support. Python 3.0, released in 2008, was a major revision that is not completely backward-compatible with earlier versions. Python 2 was discontinued with version 2.7.18 in 2020.</p><p>Python cons

In [ ]:
print(page_py.links)

In [11]:
import wikipedia

In [13]:
pypage = wikipedia.page("Python_(programming_language)")

In [18]:
print(pypage.content[0:2000])

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.Python is dynamically-typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000 and introduced new features such as list comprehensions, cycle-detecting garbage collection, reference counting, and Unicode support. Python 3.0, released in 2008, was a major revision that is not completely backward-compatible with earlier versions. Python 2 was discontinued with version 2.7.18 in 2020.Python consistently ranks as one of the most popular programming languag

In [19]:
print(pypage.links)

['"Hello, World!" program', '3ds Max', '?:', 'ABC (programming language)', 'ADMB', 'ALGOL', 'ALGOL 68', 'APL (programming language)', 'Abaqus', 'Academic Free License', 'Academic conference', 'Action selection', 'Activation function', 'Ada (programming language)', 'Advanced Simulation Library', 'Adversarial machine learning', 'AlexNet', 'Alex Graves (computer scientist)', 'Alex Martelli', 'Algebra', 'AlphaFold', 'AlphaGo', 'AlphaZero', 'Alternative terms for free software', 'Amazon (company)', 'AmigaOS 4', 'Amoeba (operating system)', 'Anaconda (installer)', 'Analyse-it', 'Andrew Ng', 'Android (operating system)', 'Anonymous function', 'Apache Groovy', 'Apache License', 'Apache webserver', 'Aphorism', 'Apple M1', 'Apple Public Source License', 'ArXiv (identifier)', 'Arbitrary-precision arithmetic', 'ArcGIS', 'Arithmetic operations', 'Array index', 'Array slicing', 'Artificial intelligence', 'Artificial neural network', 'Artistic License', 'Aspect-oriented programming', 'Assembly langua

Now without an API to get the HTML anchors.

In [4]:
import requests
from bs4 import BeautifulSoup

In [65]:
def get_summary_links(url):
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the element or elements that contain the summary
    summary_element = soup.find(class_='mw-parser-output')

    # cleaning
    to_clean_divs = soup.find_all('div', class_='thumb tright') + soup.find_all('div', class_='shortdescription') + soup.find_all('div', class_='hatnote') + soup.find_all('style') + soup.find_all('link')
    for div in to_clean_divs:
        div.decompose()

    # Find the div element with the id "toc"
    toc_div = soup.find(id='toc')
    preceding_elements = toc_div.find_previous_siblings()

    links = []
    offset = 0
    for element in reversed(preceding_elements):
        element_links = element.find_all('a', recursive=False)
        links.extend([(offset, link) for link in element_links])
        offset += len(element.get_text())

    return [(offset+link.parent.get_text().find(link.get_text()), link.get('title')) for (offset, link) in links]

In [67]:
get_summary_links("https://en.wikipedia.org/wiki/Matrix_(mathematics)")

[(3, 'Mathematics'),
 (48, 'Rectangle'),
 (60, 'Array variable'),
 (78, 'Number'),
 (87, 'Symbol (formal)'),
 (99, 'Expression (mathematics)'),
 (171, 'Mathematical object'),
 (549, 'Linear maps'),
 (597, 'Linear algebra'),
 (705, 'Operation (mathematics)'),
 (795, 'Matrix multiplication'),
 (828, 'Function composition'),
 (940, 'Graph theory'),
 (957, 'Incidence matrix'),
 (981, 'Adjacency matrix'),
 (1158, 'Square matrices'),
 (1306, 'Noncommutative ring'),
 (1398, 'Determinant'),
 (1554, 'Invertible matrix'),
 (1618, 'Eigenvalue'),
 (1668, 'Polynomial'),
 (1695, 'Geometry'),
 (1762, 'Geometric transformation'),
 (1801, 'Rotation (mathematics)'),
 (1816, 'Coordinate change'),
 (1839, 'Numerical analysis')]